In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import sys,os
import cv2
import caffe
net_file = '/home/zhongxuexian/ADAS_caffe/fabu/models/lane_od/400_640_ti/lane_od_deploy.prototxt'
caffe_model = '/home/zhongxuexian/ADAS_caffe/fabu/models/lane_od/400_640_ti/lane_od.caffemodel'
test_dir = '/home/zhongxuexian/20180814.avi'

if not os.path.exists (caffe_model):
    print "caffemodel does not exist"
    exit()

net = caffe.Net(net_file, caffe_model, caffe.TEST)
feature_file=open('fea_map0814_demo.txt','w')
cap = cv2.VideoCapture(test_dir)
arr=[1,2,5,7,9,10,22]
ret,frame=cap.read()
cnt=0
while ret==True:
    img = frame
    mean = (128, 128, 128)
    img = (img - mean)
    img = img.astype(np.float32)
    img = img.transpose((2,0,1))
    
    net.blobs['data'].data[...] = img[:,80:480,:]
    out = net.forward()
    feat_name = 'conv1a'
    feat_data = net.blobs[feat_name].data
    detect_out_name = 'detection_out'
    bbox_data = net.blobs[detect_out_name].data
    bbox = np.empty([bbox_data.shape[2],4],dtype=int)
    k=0
    for i in arr:
        for j in range(0,feat_data.shape[2]):
            for k in range(0,feat_data.shape[3]):
                feature_file.write(str(feat_data[0,i,j,k]))
                feature_file.write(' ')
            feature_file.write('\n')
    ret,frame=cap.read()
    cnt+=1
    if cnt>10:
        break

feature_file.close()
